In [1]:
# Imports
from google.colab import files
import io
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import joblib

import time
from sklearn.model_selection import KFold


In [2]:
# Google drive
from google.colab import drive
drive.mount('/content/gdrive')

# Change current working directory
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Thesis/')


Mounted at /content/gdrive


In [3]:
# Read data
df = pd.read_csv('Simulated_Cashflow_Lit.txt',sep=';')

In [4]:
df.head(20)

,ClNr,LoB,cc,AY,AQ,age,inj_part,RepDel,Pay00,Pay01,Pay02,Pay03,Pay04,Pay05,Pay06,Pay07,Pay08,Pay09,Pay10,Pay11,Open00,Open01,Open02,Open03,Open04,Open05,Open06,Open07,Open08,Open09,Open10,Open11,Lit00,Lit01,Lit02,Lit03,Lit04,Lit05,Lit06,Lit07,Lit08,Lit09,Lit10,Lit11,UC
0,1.0,2.0,49.0,1.0,1.0,32.0,42.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,2.0,2.0,46.0,1.0,1.0,15.0,18.0,0.0,651.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651.00
2,3.0,3.0,50.0,1.0,0.0,39.0,4.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,4.0,2.0,41.0,1.0,2.0,36.0,17.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,5.0,3.0,20.0,1.0,0.0,59.0,31.0,0.0,4455.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4455.00
5,6.0,1.0,6.0,1.0,0.0,21.0,7.0,0.0,430.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,430.00
6,7.0,1.0,27.0,1.0,0.0,46.0,20.0,0.0,237.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,237.00
7,8.0,4.0,46.0,1.0,3.0,51.0,19.0,0.0,0.0,257.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.00
8,9.0,4.0,26.0,1.0,3.0,36.0,8.0,0.0,0.0,628.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,628.00
9,10.0,2.0,40.0,1.0,3.0,33.0,29.0,0.0,0.0,2553.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2553.75


In [18]:
df_arr = np.array(df)

In [19]:
zeroh = np.sum(df_arr[:,8:20],axis = 1)

len(np.where(zeroh == 0)[0])

142502

In [20]:
closed = df_arr[:,20]
len(np.where(closed == 0)[0])

272450

In [10]:
len(df_arr)

497549

**guessing no development**

In [21]:
def truncate(df):
  """takes in data array and truncates it for fitting GBM 
  test data is 2005 data and train data is from 2004
  """
  nfeatures = 7

  # FEATUREWARNING
  #this assume years range from [1,12]. ahnd AY appens at column ind 3 and RepDel happens at column ind 7
  AY = df[:,3]  
  RepDel = df[:,7]
  
  # FIX TEST DATA (2015)
  dates = (12-AY-RepDel).astype(int)
  idx = np.where(dates <= 10)[0]
  X_test = np.zeros(len(idx))
  y_test = np.zeros(len(idx))

  y_test = df[idx, -1]
  for i,k in enumerate(idx):
    date = dates[k]
    X_test[i] = np.sum(df[k,8:9+date])

  # FIX TRAIN DATA (2014)
  dates = (12-AY-RepDel-1).astype(int)
  idx = np.where(dates >= 0)[0]

  X_train = np.zeros(len(idx))
  y_train = np.zeros(len(idx))

  y_train = df[idx, -1]
  for i,k in enumerate(idx):
    date = dates[k]
    X_train[i] = np.sum(df[k,8:9+date])

  return X_train, X_test, y_train, y_test


In [23]:
X_train, X_test, y_train, y_test = truncate(df_arr)

In [29]:
kf = KFold(n_splits=5, shuffle=True,random_state = 0)
val_RMSE = []

for train_index, val_index in kf.split(X_train):
  xx = X_train[val_index]
  yy = y_train[val_index]
  RMSE = mean_squared_error(xx,yy,squared = False)
  val_RMSE.append(RMSE)

print("val errors: ",val_RMSE)
test_error = mean_squared_error(X_test,y_test,squared = False)
print("test errors: ",test_error)

val errors:  [10681.066964109834, 8423.86774216109, 5681.119648091903, 6111.810273616911, 5624.002259857175]
test errors:  6673.845291138893


In [27]:
X_test

array([   0. ,    0. , 1042.5, ...,  746. , 1580. ,  111. ])

In [28]:
y_test

array([   0. ,    0. , 1042.5, ...,  746. , 1580. ,  111. ])